In [ ]:
#go into the folder you want, initialize an empty repo
!git init

In [ ]:
#Add the files you want to push. Use add . if you wanna add all the files.
!git add PPEDetection.ipynb


In [ ]:
!git add ppe.pt

In [ ]:
#Commit all the files you have added. Also write down a message
!git commit -m "updated the code"

In [ ]:
#Create a branch called main
!git branch -M main


In [ ]:
#git remote add origin into the github repo you want.
!git remote add origin https://github.com/Ngonidzashe-ux/PPEDetectionSystem.git

In [ ]:
#Push the changes to the main branch
!git push -u origin main

In [ ]:
#Check the status of your commits
!git status

In [ ]:
!git pull 

## 1. Import the required modules

In [ ]:
import cv2
from ultralytics import YOLO
import os
import math
import cvzone

## 2. Define Model Instance




In [ ]:
#Create our YOLO instance by passing in the custom trained model.
model = YOLO('ppe.pt')
classNames = ['Excavator', 'Gloves', 'Hardhat', 'Ladder', 'Mask', 'NO-Hardhat', 'NO-Mask', 'NO-Safety Vest', 'Person', 'SUV', 'Safety Cone', 'Safety Vest', 'bus', 'dump truck', 'fire hydrant', 'machinery', 'mini-van', 'sedan', 'semi', 'trailer', 'truck and trailer', 'truck', 'van', 'vehicle', 'wheel loader']


## 3. Capture Video and Classify



In [ ]:
#create a webcam object
#cap = cv2.VideoCapture(0)

#Pass a video
cap = cv2.VideoCapture('../Videos/ppe1.mp4')

myColor = (0,0,255) #Blue

while True:
    
    #read a frame and pass it into the model
    success, img = cap.read()
    results = model(img, stream=True)
    for r in results:
        boxes=r.boxes
        for box in boxes:
            
            #Get the coordinates of the 4 corners of the box
            x1,y1,x2,y2 = box.xyxy[0]
            
            #convert the coordinates from tensor type to int type
            x1,x2,y1,y2 =int(x1),int(x2),int(y1),int(y2)
            
            #debugging
            print(x1,y1,x2, y2)
            
            #x1,y1,w,h
            bbox = x1,y1, x2-x1, y2-y1
            
            #Confidence value that is in the tensor type and we round it up to the ceil and express in 2 dp
            conf = math.ceil((box.conf[0]*100))/100
            
            cls = int(box.cls[0])
            currentClass = classNames[cls]
            if conf > 0.5:
                if currentClass == 'NO-Hardhat' or currentClass == 'NO-Safety Vest' or currentClass == 'NO-Mask' or currentClass == 'NO-Gloves':
                    myColor = (0,0,255) #Red
                elif currentClass == 'Hardhat' or currentClass == 'Safety Vest' or currentClass == 'Mask' or currentClass == 'Gloves':
                    myColor = (0,255,0) #Green
                else:
                    myColor = (255,0,0) #Blue

            #put the classname, label and bounding box
            cvzone.putTextRect(img, "{} {}".format(classNames[cls], conf), (max(0,x1), max(35,y1)), scale=3.5, offset=5, thickness=1, colorB=myColor, colorT=(255, 255, 255), colorR=myColor)
            cv2.rectangle(img, (x1,y1), (x2,y2), myColor, 3)

    window_name = "Image"
    
    #display the frame img in a new window labeled 'Image'
    cv2.imshow(window_name, img)
    
    #move the window to my primary monitor
    cv2.moveWindow(window_name, 0, 0)
     
    #Removes window 2s after key pressing
    cv2.waitKey(2)
        